# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")

In [2]:
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
query = {"name":"Babelgum"}

list(db["companies"].find((query),{"name":1,"_id":0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [83]:
query2 = {"number_of_employees":{"$gt":5000}}

list(db["companies"].find((query2),{"name":1,"number_of_employees":1,"_id":0}).sort("number_of_employees", -1).limit(20))[:3]

# No entiendo por qué los datos que me aparecen no están estructurados igual que en compass, por lo que 
# saco solo dos campos.

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [114]:
query3 = {"$and":[{"founded_year":{"$gte":2000}},{"founded_year":{"$lte":2005}}]}

print(len(list(db["companies"].find((query3),{"name":1,"founded_year":1,"_id":0}))))
list(db["companies"].find((query3),{"name":1,"founded_year":1,"_id":0}))[:3]

3734


[{'founded_year': 2005, 'name': 'Wetpaint'},
 {'founded_year': 2005, 'name': 'Omnidrive'},
 {'founded_year': 2005, 'name': 'Zoho'}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [113]:
query4 = {"$and":[{"ipo.valuation_amount":{"$gt":100000000}},{"founded_year":{"$lt":2010}}]}

print(len(list(db["companies"].find((query4),{"name":1,"ipo":1,"_id":0}))))
list(db["companies"].find((query4),{"name":1,"ipo":1,"_id":0}))[:2]

42


[{'ipo': {'pub_day': 7,
   'pub_month': 11,
   'pub_year': 2013,
   'stock_symbol': 'NYSE:TWTR',
   'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD'},
  'name': 'Twitter'},
 {'ipo': {'pub_day': 18,
   'pub_month': 5,
   'pub_year': 2012,
   'stock_symbol': 'NASDAQ:FB',
   'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD'},
  'name': 'Facebook'}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [84]:
query5 = {"$and":[{"number_of_employees":{"$lt":1000}},{"founded_year":{"$lt":2005}}]}

list(db["companies"].find((query5),{"founded_year":1,"number_of_employees":1,"_id":0}).sort("number_of_employees", -1).limit(10))

[{'founded_year': 2000, 'number_of_employees': 974},
 {'founded_year': 1997, 'number_of_employees': 948},
 {'founded_year': 1997, 'number_of_employees': 931},
 {'founded_year': 1983, 'number_of_employees': 903},
 {'founded_year': 1991, 'number_of_employees': 900},
 {'founded_year': 1999, 'number_of_employees': 900},
 {'founded_year': 1999, 'number_of_employees': 881},
 {'founded_year': 2004, 'number_of_employees': 800},
 {'founded_year': 2000, 'number_of_employees': 800},
 {'founded_year': 2003, 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [35]:
query6 = {"partners":{"$exists": "False"}}

len(list(db["companies"].find(query6)))

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [112]:
query7 = {"category_code":{"$type":"null"}}

print(len(list(db.companies.find((query7),{"name":1,"category_code":1,"_id":0}))))
list(db.companies.find((query7),{"name":1,"category_code":1,"_id":0}))[:3]

# En compass con {"category_code":"null"} es suficiente, con pymongo no funciona, hay que incluir el type.

2751


[{'category_code': None, 'name': 'Collective'},
 {'category_code': None, 'name': 'Snimmer'},
 {'category_code': None, 'name': 'KoolIM'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [7]:
query8 = {"$and":[{"number_of_employees":{"$gte":100}},{"number_of_employees":{"$lt":1000}}]}

print(len(list(db["companies"].find((query8),{"name":1,"number_of_employees":1,"_id":0}))))
list(db.companies.find((query8),{"name":1,"number_of_employees":1,"_id":0}))[:3]

917


[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305}]

### 9. Order all the companies by their IPO price in a descending order.

In [110]:
query9 = {"ipo.valuation_amount":{"$exists":"true"}}

print(len(list(db["companies"].find((query9),{"name":1,"ipo.valuation_amount":1,"_id":0}).sort("ipo.valuation_amount", -1))))
list(db.companies.find((query9),{"name":1,"ipo.valuation_amount":1,"_id":0}).sort("ipo.valuation_amount", -1))[:3]

390


[{'ipo': {'valuation_amount': 108960000000}, 'name': 'GREE'},
 {'ipo': {'valuation_amount': 104000000000}, 'name': 'Facebook'},
 {'ipo': {'valuation_amount': 100000000000}, 'name': 'Amazon'}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [90]:
query10 = {"number_of_employees":{"$exists":"true"}}

list(db.companies.find((query10),{"name":1,"number_of_employees":1,"_id":0}).sort("number_of_employees", -1).limit(10))[:3]

# Mismo caso que el ejercicio 2

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [91]:
query11 = {"founded_month":{"$gt":6}}

list(db.companies.find((query11),{"name":1,"founded_month":1,"_id":0}).limit(1000))[:3]

[{'founded_month': 10, 'name': 'Wetpaint'},
 {'founded_month': 11, 'name': 'Omnidrive'},
 {'founded_month': 9, 'name': 'Zoho'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [108]:
query12 = {"$and":[{"founded_year":{"$lt":2000}},{"acquisition.price_amount":{"$gt":10000000}}]}

print(len(list(db["companies"].find((query12),{"founded_year":1,"acquisition.price_amount":1,"_id":0}))))
list(db.companies.find((query12),{"founded_year":1,"acquisition.price_amount":1,"_id":0}))[:3]

205


[{'acquisition': {'price_amount': 625000000}, 'founded_year': 1999},
 {'acquisition': {'price_amount': 180000000}, 'founded_year': 1999},
 {'acquisition': {'price_amount': 25000000}, 'founded_year': 1999}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [107]:
query13 = {"acquisition.acquired_year":{"$gt":2010}}

print(len(list(db["companies"].find((query13),{"name":1,"acquisition":1,"_id":0}).sort("acquisition.price_amount",-1))))
list(db.companies.find((query13),{"name":1,"acquisition":1,"_id":0}).sort("acquisition.price_amount",-1))[:1]

736


[{'acquisition': {'acquired_day': 20,
   'acquired_month': 3,
   'acquired_year': 2011,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'},
   'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'term_code': None},
  'name': 'T-Mobile'}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [109]:
query14 = {"founded_year":{"$exists":"true"}}

list(db.companies.find((query14),{"name":1,"founded_year":1,"_id":0}).sort("founded_year", -1).limit(1000))[:3]

[{'founded_year': 2013, 'name': 'Fixya'},
 {'founded_year': 2013, 'name': 'Wamba'},
 {'founded_year': 2013, 'name': 'Advaliant'}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [95]:
query15 = {"founded_day":{"$lte":7}}

list(db.companies.find((query15),{"founded_day":1,"acquisition.price_amount":1,"_id":0}).sort("acquisition.price_amount", -1).limit(10))

[{'acquisition': {'price_amount': 4200000000}, 'founded_day': 4},
 {'acquisition': {'price_amount': 1500000000}, 'founded_day': 1},
 {'acquisition': {'price_amount': 1200000000}, 'founded_day': 1},
 {'acquisition': {'price_amount': 1000000000}, 'founded_day': 1},
 {'acquisition': {'price_amount': 625000000}, 'founded_day': 2},
 {'acquisition': {'price_amount': 500000000}, 'founded_day': 1},
 {'acquisition': {'price_amount': 410000000}, 'founded_day': 6},
 {'acquisition': {'price_amount': 345000000}, 'founded_day': 1},
 {'acquisition': {'price_amount': 321000000}, 'founded_day': 1},
 {'acquisition': {'price_amount': 300000000}, 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [103]:
query16 = {"$and":[{"category_code":"web"},{"number_of_employees":{"$gt":4000}}]}

print(len(list(db["companies"].find((query16),{"category_code":1,"number_of_employees":1,"_id":0}).sort("number_of_employees", 1))))
list(db.companies.find((query16),{"category_code":1,"number_of_employees":1,"_id":0}).sort("number_of_employees", 1))[:3]

9


[{'category_code': 'web', 'number_of_employees': 4400},
 {'category_code': 'web', 'number_of_employees': 8000},
 {'category_code': 'web', 'number_of_employees': 8657}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [97]:
query17 = {"$and":[{"acquisition.price_amount":{"$gt":10000000},"acquisition.price_currency_code":"EUR"}]}

list(db.companies.find((query17),{"acquisition.price_amount":1,"acquisition.price_currency_code":1,"_id":0}))

[{'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [99]:
query18 = {"acquisition.acquired_month":{"$lt":4}}

list(db.companies.find((query18),{"name":1,"acquisition":1,"_id":0}).limit(10))[:2]

[{'acquisition': {'acquired_day': 31,
   'acquired_month': 1,
   'acquired_year': 2011,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'},
   'price_amount': None,
   'price_currency_code': 'USD',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'term_code': None},
  'name': 'Kyte'},
 {'acquisition': {'acquired_day': None,
   'acquired_month': 2,
   'acquired_year': 2007,
   'acquiring_company': {'name': 'Nielsen', 'permalink': 'nielsen'},
   'price_amount': 327000000,
   'price_currency_code': 'USD',
   'source_description': 'Nielsen buys rest of NetRatings',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [6]:
query19 = {"$and":[{"founded_year":{"$gte":2000},"founded_year":{"$lte":2010},"acquisitions.0.acquired_year":{"$gt":2011},"acquisitions.1.acquired_year":{"$gt":2011}}]}

# Hay que tener en cuenta que puede haber dos acquisitions

print(len(list(db["companies"].find((query19),{"founded_year":1,"acquisitions.acquired_year":1,"_id":0}))))
list(db.companies.find((query19),{"founded_year":1,"acquisitions.acquired_year":1,"_id":0}))[:3]

34


[{'acquisitions': [{'acquired_year': 2012}, {'acquired_year': 2012}],
  'founded_year': 2008},
 {'acquisitions': [{'acquired_year': 2012}, {'acquired_year': 2013}],
  'founded_year': 2005},
 {'acquisitions': [{'acquired_year': 2012}, {'acquired_year': 2013}],
  'founded_year': 2004}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [3]:
db.companies.aggregate([{"$project":{"name":1,"diference":{"$substract":["deadpooled_year","founded_year"]}}}])

OperationFailure: Unrecognized expression '$substract'